# Biodiversidad
Script para calcular riqueza de especies en un polígono dado, utilizando la API de GBIF

En primer lugar, instalamos las bibliotecas que se van a utilizar en el proceso

In [42]:
pip install geopandas shapely pygbif

Ahora cargamos las bibliotecas

In [43]:
import geopandas as gpd
from pygbif import occurrences
from shapely.geometry import mapping
import json
import requests

In [52]:
# 1. Cargar y preparar el polígono
ruta_archivo_geojson = '/content/poligono_prueba_2.geojson'
gdf = None # Initialize gdf
wkt = None # Initialize wkt

try:
    gdf = gpd.read_file(ruta_archivo_geojson)

    # Asegurar que esté en WGS84 (EPSG:4326) que es lo que usa GBIF
    if gdf.crs != "EPSG:4326":
        print("Reproyectando polígono a WGS84...")
        gdf = gdf.to_crs("EPSG:4326")

    # Obtener la geometría WKT (Well Known Text) del primer polígono
    # GBIF requiere WKT para búsquedas espaciales
    geom = gdf.geometry.iloc[0]
    wkt = geom.wkt
    print("Polígono cargado y WKT generado con éxito.")

except Exception as e:
    print(f"Error cargando o procesando el polígono: {e}")

Polígono cargado y WKT generado con éxito.


In [53]:
if 'gdf' in locals():
    m = gdf.explore(tiles='OpenStreetMap', style_kwds=dict(color='red', fillOpacity=0.2, weight=2))
    display(m)
else:
    print("El GeoDataFrame 'gdf' no ha sido creado. Por favor, asegúrate de ejecutar las celdas anteriores.")

In [54]:
def get_gbif_taxon_key(name, rank=None):
    url = "https://api.gbif.org/v1/species/match"
    params = {"name": name}
    if rank:
        params["rank"] = rank
    r = requests.get(url, params=params)
    data = r.json()
    return data.get("usageKey")

# Definir grupos con nombre + rango; sin taxonKey fijo
grupos = {
    "Aves": {"name": "Aves", "rank": "CLASS"},
    "Mamíferos": {"name": "Mammalia", "rank": "CLASS"},
    "Reptiles": {"name": "Reptilia", "rank": "CLASS"},
    "Anuros": {"name": "Anura", "rank": "ORDER"},
    "Plantas vasculares": {"name": "Tracheophyta", "rank": "PHYLUM"},
}

# Obtener taxonKeys actualizados
for grupo, info in grupos.items():
    key = get_gbif_taxon_key(info["name"], rank=info["rank"])
    info["taxonKey"] = key

print("Grupos con taxonKeys actualizados:")
print(grupos)

# Luego puedes usar grupos[...] ["taxonKey"] en tu consulta a GBIF

Grupos con taxonKeys actualizados:
{'Aves': {'name': 'Aves', 'rank': 'CLASS', 'taxonKey': 212}, 'Mamíferos': {'name': 'Mammalia', 'rank': 'CLASS', 'taxonKey': 359}, 'Reptiles': {'name': 'Reptilia', 'rank': 'CLASS', 'taxonKey': 44}, 'Anuros': {'name': 'Anura', 'rank': 'ORDER', 'taxonKey': 952}, 'Plantas vasculares': {'name': 'Tracheophyta', 'rank': 'PHYLUM', 'taxonKey': 7707728}}


In [56]:
import geopandas as gpd
from shapely.ops import orient # <--- IMPORTANTE: Agrega esto al inicio

# ... (código de carga del archivo) ...

# 1. Obtener geometría
geom = gdf.geometry.iloc[0]

# 2. Simplificar si es necesario (para evitar URLs kilométricas)
# Usamos una tolerancia fina para no deformar mucho el polígono
if len(geom.exterior.coords) > 500:
    geom = geom.simplify(tolerance=0.0001, preserve_topology=True)

# 3. CORRECCIÓN DEL ERROR 400: Forzar orientación
# sign=1.0 asegura sentido Anti-Horario (Counter-Clockwise)
geom_oriented = orient(geom, sign=1.0)

# 4. Generar el WKT
wkt = geom_oriented.wkt

print("Geometría orientada y lista.")
# ... (continuar con el bucle for) ...

Geometría orientada y lista.


In [57]:
# 3. Iterar por cada grupo y consultar la API
for nombre_grupo, data in grupos.items():
        key = data['taxonKey']

        # Hacemos una búsqueda facetada.
        # limit=0: No queremos descargar las observaciones individuales (ahorra tiempo).
        # facet='speciesKey': Queremos que agrupe por especies.
        # facetLimit=100000: Intentamos traer todas las especies posibles (aumentar si el área es gigante).

        try:
            res = occurrences.search(
                geometry=wkt,
                taxonKey=key,
                hasCoordinate=True, # Solo datos con coordenadas
                limit=0,
                facet='speciesKey',
                facetLimit=20000 # Límite de seguridad, ajustable
            )

            # El número de buckets en la faceta 'speciesKey' equivale a la riqueza de especies
            # Nota: Esto cuenta 'speciesKey' únicos que tienen al menos 1 registro.
            facets = res.get('facets', [])
            riqueza = 0

            for f in facets:
                if f['field'] == 'SPECIES_KEY':
                    riqueza = len(f['counts'])
                    break

            resultados[nombre_grupo] = riqueza
            print(f"✅ {nombre_grupo}: {riqueza} especies encontradas.")

        except Exception as e:
            print(f"❌ Error consultando {nombre_grupo}: {e}")
            resultados[nombre_grupo] = "Error"

✅ Aves: 229 especies encontradas.
✅ Mamíferos: 12 especies encontradas.
✅ Reptiles: 254 especies encontradas.
✅ Anuros: 8 especies encontradas.
✅ Plantas vasculares: 120 especies encontradas.
